<div class="alert alert-block alert-info">
     <b>Import Modules Needed</b> <i>(need pandas >= 1.2 and python >= 3.9.0 for all the data cleaning techniques to work)</i>
</div>

In [1]:
import pandas as  pd
import os
import sweetviz as sv
import numpy as np
import dtale
import cudf
import dask_cudf

#!pip install git+https://github.com/innovationOUtside/fstring-magic.git

<div class="alert alert-block alert-info">
    <b>Jupyter Plumbing</b>
</div>


In [2]:
# %load C:\users\derek\.jupyter\startup.py
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#pd.set_option('display.max_colwidth', 100)

pd.set_option('display.max_colwidth', None)



pd.set_option("display.max_rows", None)

pd.options.display.max_columns = 500

pd.options.display.max_seq_items = 500



<div class="alert alert-block alert-info">
    <b>Find CSV Files in working dir</b>
</div>

In [3]:
for file in os.listdir():
    if file.endswith(".csv"):
        print(os.path.join("/", file))

/df_visits.csv
/df_episode.csv
/final.csv
/df_OASIS.csv
/dead_patients_final.csv
/orig_data2.csv
/eliminated.csv
/orig_data.csv
/cleaned_data.csv
/event_mapping_df.csv
/du.csv
/36693_135410.csv
/df_episodes3.csv
/36693_135410_episode.csv
/df_episodes4.csv
/patients.csv
/soc.csv
/alive_patients_final.csv
/df_episodes2.csv
/deduped_complete.csv


<div class="alert alert-block alert-info">
    <b>Import the raw data</b>
</div>

In [4]:
df = pd.read_csv('final.csv').iloc[: , 2:]


<div class="alert alert-block alert-info">
    <b>Find Columns that contain mixed data types to investigate for possible data quality issues</b>
</div>


In [5]:
if mixed_dtypes := {c: dtype for c in df.columns if (dtype := pd.api.types.infer_dtype(df[c])).startswith("mixed")}:
    raise TypeError(f"Dataframe has one more mixed dtypes: {mixed_dtypes}")

<div class="alert alert-block alert-info">
    <b>Find Columns with Dates and convert them to datetime type</b>
</div>

In [6]:
cols_w_dates = df.columns[(df.columns.str.contains('DT', case=False) | df.columns.str.contains('date', case=False) | df.columns.str.contains('OFEPISODE', case=False))] 
df[cols_w_dates] = df[cols_w_dates].apply(pd.to_datetime)
cols_w_dates


Index(['epi_SocDate', 'epi_StartOfEpisode', 'epi_EndOfEpisode',
       'epi_DischargeDate', 'M0906_DC_TRAN_DTH_DT', 'M0090_INFO_COMPLETED_DT',
       'cedd_dateofdeath'],
      dtype='object')

<div class="alert alert-block alert-info">
    <b>Break up columns</b>
</div>

In [7]:
# Descriptive IDs
df.columns[:28]
colsMeta = df.columns[:26]
# OASIS Factors
df.columns[28:]
colsOASIS = df.columns[26:]

Index(['DispositionId', 'mortality', 'pa_id', 'year_born', 'pa_gender',
       'epi_id', 'ceo_id', 'epi_SocDate', 'epi_StartOfEpisode',
       'epi_EndOfEpisode', 'epi_DischargeDate', 'M0906_DC_TRAN_DTH_DT',
       'M0090_INFO_COMPLETED_DT', 'cedd_dateofdeath', 'DaysToDeath',
       'epi_AdmitType', 'dcc_desc', 'epi_dcrid', 'dcr_desc', 'ceo_HIPPS',
       'ceo_HHRG', 'ceas_ascode', 'icd_code', 'icd_code_description',
       'DiagnosisCategory', 'RoughAge', 'ceo_OASIScorrectionnum',
       'M0100_ASSMT_REASON'],
      dtype='object')

Index(['M0110_EPISODE_TIMING', 'M0140_ETHNIC_AI_AN', 'M0140_ETHNIC_ASIAN',
       'M0140_ETHNIC_BLACK', 'M0140_ETHNIC_HISP', 'M0140_ETHNIC_NH_PI',
       'M0140_ETHNIC_WHITE', 'M1000_DC_IPPS_14_DA', 'M1000_DC_IRF_14_DA',
       'M1000_DC_LTC_14_DA', 'M1000_DC_LTCH_14_DA', 'M1000_DC_NONE_14_DA',
       'M1000_DC_OTH_14_DA', 'M1000_DC_PSYCH_14_DA', 'M1000_DC_SNF_14_DA',
       'M1028_ACTV_DIAG_DM', 'M1028_ACTV_DIAG_NOA', 'M1028_ACTV_DIAG_PVD_PAD',
       'M1030_THH_ENT_NUTRITION', 'M1030_THH_IV_INFUSION',
       'M1030_THH_NONE_ABOVE', 'M1030_THH_PAR_NUTRITION',
       'M1033_HOSP_RISK_5PLUS_MDCTN', 'M1033_HOSP_RISK_COMPLIANCE',
       'M1033_HOSP_RISK_CRNT_EXHSTN', 'M1033_HOSP_RISK_HSTRY_FALLS',
       'M1033_HOSP_RISK_MLTPL_ED_VISIT', 'M1033_HOSP_RISK_MLTPL_HOSPZTN',
       'M1033_HOSP_RISK_MNTL_BHV_DCLN', 'M1033_HOSP_RISK_NONE_ABOVE',
       'M1033_HOSP_RISK_OTHR_RISK', 'M1033_HOSP_RISK_WEIGHT_LOSS',
       'M1051_PVX_RCVD_AGNCY', 'M1060_HEIGHT_A', 'M1060_WEIGHT_B',
       'M1200_VISI

<div class="alert alert-block alert-info">
    <b>Important Summary Stats</b>
</div>

In [8]:
f"There are {df.groupby('DispositionId').agg({'pa_id': 'nunique'}).sum().values[0]} patients"

f"Alive: {df.groupby('mortality').agg({'pa_id': 'nunique'}).iloc[0,0]} | Dead: {df.groupby('mortality').agg({'pa_id': 'nunique'}).iloc[1,0]}"

'There are 1918 patients'

'Alive: 159 | Dead: 1759'

<hr style="height:2px;border-width:0;color:black;background-color:black">

<div class="alert alert-info" role="alert">
    <b>Create a patients df and populate the death date by taking the most recent dates</b>
</div>

In [9]:
len(df.groupby(['pa_id'])['pa_id','DispositionId','pa_gender','year_born', 'M0906_DC_TRAN_DTH_DT', 'mortality'].transform('max').drop_duplicates('pa_id'))

df_patients = df.groupby(['pa_id'])['pa_id', 'DispositionId' , 'pa_gender','year_born', 'M0906_DC_TRAN_DTH_DT', 'mortality', 'cedd_dateofdeath'].transform('max').drop_duplicates('pa_id').reset_index(drop=True)

1918

<div class="alert alert-info" role="alert">
    <b>If no cedd_dateofdeath is provided then use the 'M0906_DC_TRAN_DTH_DT' </b>
</div>

In [10]:
df_patients['cedd_dateofdeath'].where(((df_patients.mortality == 0) | (~df_patients.cedd_dateofdeath.isnull())), (df_patients['M0906_DC_TRAN_DTH_DT']), inplace=True)

<div class="alert alert-info" role="alert">
    <b>Create a Year of Death + Age at Death column</b>
</div>

In [11]:
df_patients['yod']= pd.DatetimeIndex(df_patients['cedd_dateofdeath']).year
df_patients['aod'] = df_patients['yod'] - df_patients['year_born']


<div class="alert alert-info" role="alert">
    <b>Convert YOD and AAD into integers</b>
</div>

In [12]:
#df_patients.fillna({'cedd_dateofdeath':0,}, inplace=True)
df_patients.fillna({'yod':0,}, inplace=True)
df_patients["yod"] = df_patients["yod"].apply(np.int64)
df_patients.fillna({'aod':0,}, inplace=True)
df_patients['aod'] = df_patients['aod'].apply(np.int64)

In [13]:
dtale.show(df_patients)

<div class="alert alert-info" role="alert">
    <b>Export df_patients to .csv</b>
</div>

In [14]:
df_patients.to_csv('patients.csv', sep=',', encoding='utf-8', index=False)

<hr style="height:2px;border-width:0;color:black;background-color:black">

<div class="alert alert-warning" role="alert">
    <b>Find Best Columns to Use for Event Data </b>
</div>

In [15]:
(df.loc[:,colsOASIS][df['mortality']==0].isnull().sum()/len(df.loc[:,colsOASIS][df['mortality']==0])).sort_values(ascending=False)
#mask2 = (df.loc[:,colsOASIS][df['DispositionId']==2].isnull().sum()/len(df.loc[:,colsOASIS][df['DispositionId']==2])>=.75).sort_values(ascending=False)

M1060_WEIGHT_B                    0.955335
M1060_HEIGHT_A                    0.952854
M1730_PHQ2_LACK_INTRST            0.920596
M1730_PHQ2_DPRSN                  0.920596
M1000_DC_OTH_14_DA                0.913151
M1910_MLT_FCTR_FALL_RISK_ASMT     0.913151
M2001_DRUG_RGMN_RVW               0.913151
M2010_HIGH_RISK_DRUG_EDCTN        0.913151
M1730_STDZ_DPRSN_SCRNG            0.913151
M1028_ACTV_DIAG_PVD_PAD           0.913151
M1028_ACTV_DIAG_NOA               0.913151
M1000_DC_SNF_14_DA                0.913151
M1000_DC_PSYCH_14_DA              0.913151
M1028_ACTV_DIAG_DM                0.913151
M1000_DC_NONE_14_DA               0.913151
M0140_ETHNIC_ASIAN                0.913151
M1000_DC_LTC_14_DA                0.913151
M1000_DC_IRF_14_DA                0.913151
M1000_DC_IPPS_14_DA               0.913151
M0140_ETHNIC_WHITE                0.913151
M0140_ETHNIC_NH_PI                0.913151
M0140_ETHNIC_HISP                 0.913151
M0140_ETHNIC_BLACK                0.913151
M1000_DC_LT

In [16]:
(df.loc[:,colsOASIS][df['mortality']==1].isnull().sum()/len(df.loc[:,colsOASIS][df['mortality']==1])).sort_values(ascending=True)
#mask2 = (df.loc[:,colsOASIS][df['DispositionId']==2].isnull().sum()/len(df.loc[:,colsOASIS][df['DispositionId']==2])>=.75).sort_values(ascending=False)

ceo_OASIScorrectionnum            0.000000
M0100_ASSMT_REASON                0.000000
M1860_CRNT_AMBLTN                 0.202447
M1850_CRNT_TRNSFRNG               0.202447
M1840_CRNT_TOILTG                 0.202447
M1830_CRNT_BATHG                  0.202447
M1820_CRNT_DRESS_LOWER            0.202447
M1810_CRNT_DRESS_UPPER            0.202447
M1306_UNHLD_STG2_PRSR_ULCR        0.202447
M1242_PAIN_FREQ_ACTVTY_MVMT       0.202447
M1400_WHEN_DYSPNEIC               0.202447
M1340_SRGCL_WND_PRSNT             0.202782
M1330_STAS_ULCR_PRSNT             0.202782
M1324_STG_PRBLM_ULCER             0.202782
M1620_BWL_INCONT                  0.202950
M1800_CRNT_GROOMING               0.216357
M1710_WHEN_CONFUSED               0.383442
M1740_BD_MEM_DEFICIT              0.383442
M1740_BD_IMP_DECISN               0.383442
M1740_BD_DELUSIONS                0.383442
M1740_BD_SOC_INAPPRO              0.383442
M1740_BD_VERBAL                   0.383442
M1745_BEH_PROB_FREQ               0.383442
M1720_WHEN_

In [17]:
(df.loc[:,colsOASIS].isnull().sum()/len(df.loc[:,colsOASIS])).sort_values(ascending=True)

ceo_OASIScorrectionnum            0.000000
M0100_ASSMT_REASON                0.000000
M1860_CRNT_AMBLTN                 0.212088
M1850_CRNT_TRNSFRNG               0.212088
M1840_CRNT_TOILTG                 0.212088
M1830_CRNT_BATHG                  0.212088
M1820_CRNT_DRESS_LOWER            0.212088
M1810_CRNT_DRESS_UPPER            0.212088
M1306_UNHLD_STG2_PRSR_ULCR        0.212088
M1242_PAIN_FREQ_ACTVTY_MVMT       0.212088
M1400_WHEN_DYSPNEIC               0.212088
M1340_SRGCL_WND_PRSNT             0.212402
M1330_STAS_ULCR_PRSNT             0.212402
M1324_STG_PRBLM_ULCER             0.212402
M1620_BWL_INCONT                  0.212559
M1800_CRNT_GROOMING               0.228414
M1710_WHEN_CONFUSED               0.408163
M1740_BD_NONE                     0.408163
M1740_BD_MEM_DEFICIT              0.408163
M1740_BD_IMP_DECISN               0.408163
M1740_BD_DELUSIONS                0.408163
M1740_BD_VERBAL                   0.408163
M1745_BEH_PROB_FREQ               0.408163
M1720_WHEN_

<hr style="height:2px;border-width:0;color:black;background-color:black">

<div class="alert alert-success" role="alert">
    <b>Create an events dataframe</b>
</div>

In [10]:
cols_Oevents = ['M0100_ASSMT_REASON', 
                'M1860_CRNT_AMBLTN', 
                'M1850_CRNT_TRNSFRNG', 
                'M1830_CRNT_BATHG', 
                'M1820_CRNT_DRESS_LOWER', 
                'M1810_CRNT_DRESS_UPPER', 
                'M1306_UNHLD_STG2_PRSR_ULCR', 
                'M1242_PAIN_FREQ_ACTVTY_MVMT', 
                'M1400_WHEN_DYSPNEIC', 
                'M1340_SRGCL_WND_PRSNT', 
                'M1330_STAS_ULCR_PRSNT',
                'M1324_STG_PRBLM_ULCER', 
                'M1620_BWL_INCONT', 
                'M1800_CRNT_GROOMING'] 

<div class="alert alert-success" role="alert">
    <b>Create an Items Mapping</b>
</div>

In [11]:
event_mapping_df = pd.DataFrame(cols_Oevents,columns =['OASIS']).rename_axis('itemID').shift()[1:].reset_index(inplace=False)

In [12]:
dtale.show(event_mapping_df)

In [18]:
event_mapping_df.to_csv('event_mapping_df.csv', sep=',', encoding='utf-8', index=False)

<hr style="height:2px;border-width:0;color:black;background-color:black">

<div class="alert alert-danger" role="alert">
    <b>Extract the integer from the event descriptions</b>
</div>

In [38]:
df_events_int = df[cols_Oevents].apply(lambda x: x[cols_Oevents].str[:2].str.strip(), axis = 1).replace('NA',0)

In [20]:
df_events_desc = df[cols_Oevents].apply(lambda x: x[cols_Oevents].str[2:].str.strip(), axis = 1)

In [21]:
len(df_events_int)
df_events_desc.columns

6370

Index(['M0100_ASSMT_REASON', 'M1860_CRNT_AMBLTN', 'M1850_CRNT_TRNSFRNG',
       'M1830_CRNT_BATHG', 'M1820_CRNT_DRESS_LOWER', 'M1810_CRNT_DRESS_UPPER',
       'M1306_UNHLD_STG2_PRSR_ULCR', 'M1242_PAIN_FREQ_ACTVTY_MVMT',
       'M1400_WHEN_DYSPNEIC', 'M1340_SRGCL_WND_PRSNT', 'M1330_STAS_ULCR_PRSNT',
       'M1324_STG_PRBLM_ULCER', 'M1620_BWL_INCONT', 'M1800_CRNT_GROOMING'],
      dtype='object')

In [151]:
from math import isnan
xet = dict(zip(df_events_int.M1860_CRNT_AMBLTN, df_events_desc.M1860_CRNT_AMBLTN))

In [152]:
xet.keys()


dict_keys(['3', '2', '4', nan, '1', '5', '0', '6'])

In [157]:
clean_dict = {k: xet[k] for k in xet if not pd.isna(k)}
#clean_dict = filter(lambda k: not pd.isna(k), xet)
clean_dict



{'3': '- ABLE TO WALK ONLY WITH THE SUPERVISION OR ASSISTANCE OF ANOTHER PERSON AT ALL TIMES.',
 '2': '- REQUIRES USE OF A TWO-HANDED DEVICE (FOR EXAMPLE, WALKER OR CRUTCHES) TO WALK ALONE ON A LEVEL SURFACE AND/OR REQUIRES HUMAN SUPERVISION OR ASSISTANCE TO NEGOTIATE STAIRS OR STEPS OR UNEVEN SURFACES.',
 '4': '- CHAIRFAST, UNABLE TO AMBULATE BUT IS ABLE TO WHEEL SELF INDEPENDENTLY.',
 '1': '- WITH THE USE OF A ONE-HANDED DEVICE (FOR EXAMPLE, CANE, SINGLE CRUTCH, HEMI-WALKER), ABLE TO INDEPENDENTLY WALK ON EVEN AND UNEVEN SURFACES AND NEGOTIATE STAIRS WITH OR WITHOUT RAILINGS.',
 '5': '- CHAIRFAST, UNABLE TO AMBULATE AND IS UNABLE TO WHEEL SELF.',
 '0': '- ABLE TO INDEPENDENTLY WALK ON EVEN AND UNEVEN SURFACES AND NEGOTIATE STAIRS WITH OR WITHOUT RAILINGS (SPECIFICALLY: NEEDS NO HUMAN ASSISTANCE OR ASSISTIVE DEVICE).',
 '6': '- BEDFAST, UNABLE TO AMBULATE OR BE UP IN A CHAIR'}

In [118]:
len(df_events_desc)
df_events_desc.columns

6370

Index(['M0100_ASSMT_REASON', 'M1860_CRNT_AMBLTN', 'M1850_CRNT_TRNSFRNG',
       'M1830_CRNT_BATHG', 'M1820_CRNT_DRESS_LOWER', 'M1810_CRNT_DRESS_UPPER',
       'M1306_UNHLD_STG2_PRSR_ULCR', 'M1242_PAIN_FREQ_ACTVTY_MVMT',
       'M1400_WHEN_DYSPNEIC', 'M1340_SRGCL_WND_PRSNT', 'M1330_STAS_ULCR_PRSNT',
       'M1324_STG_PRBLM_ULCER', 'M1620_BWL_INCONT', 'M1800_CRNT_GROOMING'],
      dtype='object')

<div class="alert alert-danger" role="alert">
    <b>Combine the descriptive columns with the desired events</b>
</div>

In [39]:
colsM = ['pa_id', 'epi_id', 'ceo_id', 'M0090_INFO_COMPLETED_DT']

In [40]:
df_events = pd.concat([df[colsM], df_events_int], axis = 1).drop_duplicates().reset_index().drop('index', axis=1, errors='ignore')

In [43]:
dtale.show(df_events)

<div class="alert alert-danger alert-info">
    <b>Convert from Wide to Long format</b>
</div>

In [44]:
df_OASIS = df_events.melt(id_vars=colsM, var_name = 'OASIS', value_vars=cols_Oevents)

In [45]:
df_OASIS = df_OASIS.merge(event_mapping_df,left_on='OASIS', right_on='OASIS', how='left')

<div class="alert alert-danger alert-info">
    <b>Convert itemID to Integers (this actually caused problems later on when serializing to JSON!!</b>
</div>

In [46]:
df_OASIS.fillna({'itemID':0,}, inplace=True)
df_OASIS['itemID'] = df_OASIS['itemID'].apply(np.int64)

<div class="alert alert-danger alert-info">
    <b>Rearrange the columns to match MIMIC</b>
</div>

In [47]:
mid = df_OASIS['itemID']
mid2 = df_OASIS['value']
df_OASIS.drop(labels=['itemID', 'value'], axis=1, inplace = True)
df_OASIS.insert(4, 'itemID', mid)
df_OASIS.insert(5, 'value', mid2)

In [48]:
len(df_OASIS)
#df_OASIS.head()
dtale.show(df_OASIS.head())

88900

<div class="alert alert-danger alert-info">
    <b>Export to CSV</b>
</div>

In [50]:
df_OASIS.to_csv('df_OASIS.csv', sep=',', encoding='utf-8', index=False)

Executing shutdown due to inactivity...


2022-04-04 14:40:29,764 - INFO     - Executing shutdown due to inactivity...


Executing shutdown...


2022-04-04 14:40:29,771 - INFO     - Executing shutdown...


<hr style="height:2px;border-width:0;color:black;background-color:black">

<div class="alert alert-success" role="alert">
  Create an visits df
</div>

In [75]:
dfv = df.set_index(['pa_id', 'epi_id', 'ceo_id'])#[['M0090_INFO_COMPLETED_DT', 'M0100_ASSMT_REASON']]

In [ ]:
dtale.show(df[['M0090_INFO_COMPLETED_DT', 'M0100_ASSMT_REASON']].reset_index().drop_duplicates())

In [73]:
df_visits = dfv[['M0090_INFO_COMPLETED_DT', 'M0100_ASSMT_REASON']].reset_index().drop('index', axis=1, errors='ignore').duplicated(keep=False)

In [74]:
dtale.show(df[df_visits][['pa_id', 'epi_id', 'ceo_id','M0090_INFO_COMPLETED_DT', 'M0100_ASSMT_REASON']])

In [80]:
df_visits = dfv[['M0090_INFO_COMPLETED_DT', 'M0100_ASSMT_REASON']].reset_index().drop('index', axis=1, errors='ignore').drop_duplicates().reset_index().drop('index', axis=1, errors='ignore')

In [81]:
dtale.show(df_visits) #'epi_StartOfEpisode', 'epi_EndOfEpisode', 'epi_DischargeDate', 

In [82]:
df_visits.to_csv('df_visits.csv', sep=',', encoding='utf-8', index=False)

<hr style="height:2px;border-width:0;color:black;background-color:black">

<div class="alert alert-success" role="alert">
  Create an episodes df
</div>

In [25]:
dfe = df.set_index(['pa_id', 'epi_id'])#[['M0090_INFO_COMPLETED_DT', 'M0100_ASSMT_REASON']]

In [26]:
df_episodes = dfe[['mortality','epi_StartOfEpisode', 'epi_EndOfEpisode', 'epi_DischargeDate', 'epi_AdmitType',  'dcc_desc', 'dcr_desc', 'icd_code', 'icd_code_description']].reset_index().drop('index', axis=1, errors='ignore').drop_duplicates().reset_index().drop('index', axis=1, errors='ignore')

In [37]:
int(~pd.isna(df_episodes.epi_DischargeDate).values[0])

1

In [27]:
dtale.show(df_episodes)

In [38]:
df_episodes.to_csv('df_episode.csv', sep=',', encoding='utf-8', index=False)

Executing shutdown due to inactivity...


2022-04-01 17:02:25,867 - INFO     - Executing shutdown due to inactivity...


Executing shutdown...


2022-04-01 17:02:25,874 - INFO     - Executing shutdown...


<hr style="height:2px;border-width:0;color:black;background-color:black">